In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression, Ridge

import numpy.polynomial.polynomial as poly

In [ ]:
def get_clean_data(file):
    # Reads panadas dataframe
    df = pd.read_csv(file)
    # Convert datetime to a number
    df['SALE DATE'] = pd.to_datetime(df['SALE DATE']).astype(np.int64)
    # Drop other unneeded columns
    data = df.drop(columns = ["NEIGHBORHOOD", "APARTMENT NUMBER", "BUILDING CLASS CATEGORY", "TAX CLASS AT PRESENT", "BUILDING CLASS AT PRESENT", "EASE-MENT", "ADDRESS", "ZIP CODE", "BUILDING CLASS AT TIME OF SALE", "TOTAL UNITS"])
    # Drop other boroughs
    # data = data[(data['BOROUGH'] != 1) & (data['BOROUGH'] != 2) & (data['BOROUGH'] != 5)]
    data = data[(data['BOROUGH'] == 3) | (data['BOROUGH'] == 4)]
    # Convert all other strings to int values
    data = data.astype(int)
    # print(x.info())
    return data

In [ ]:
def preprocess_normalize_data(df):
    df_x_prescale = df[["BLOCK","LOT","RESIDENTIAL UNITS","COMMERCIAL UNITS","LAND SQUARE FEET", "GROSS SQUARE FEET","YEAR BUILT","TAX CLASS AT TIME OF SALE","SALE PRICE", "SALE DATE"]]
    df_y = df[["BOROUGH"]]
    # print(df_x_prescale)

    x_scaled_array = preprocessing.scale(df_x_prescale)
    y_array = np.array(df_y)

    df_z_scaled = df_x_prescale.copy()
    # print(df_z_scaled)
    # apply normalization technique to Sale Date
    column = 'SALE DATE'
    df_z_scaled[column] = (df_z_scaled[column] - df_z_scaled[column].mean()) / df_z_scaled[column].std()
    # print(df_z_scaled)
    
    X_train, X_test, y_train, y_test = train_test_split(x_scaled_array, y_array, test_size = 0.8, train_size = 0.2)
    return X_train, y_train

# Regular Logistic Regression

In [ ]:
def train_logistic(X_train, y_train):
    logreg = LogisticRegression(C = 100000000) # regular logistic regression, no regularization
    logreg.fit(X_train, y_train)
    return logreg.coef_, logreg.intercept_, logreg.score(X_train, y_train)

In [ ]:
data = get_clean_data("cleaned_data.csv")
# print(data)
# print(data.info(verbose=True))

# X_train, y_train = no_feature_scaling(data)
# X_train, y_train = preprocess_scale_data(data)
X_train, y_train = preprocess_normalize_data(data)
# X_train, y_train = preprocess_standard_scalar(data)

# print(X_train)
# print(y_train)

coef, inter, score = train_logistic(X_train, y_train)
print("Coefficients", coef)
print("Intercept", inter)
print("Score", score)

Coefficients [[ 9.50880458e-01 -1.44402694e+00  4.85175815e-01  5.11669711e+00
   4.05724729e-01 -8.82870072e-01  2.20630365e+00 -5.51493602e-01
   8.38500332e-02  4.26622461e-03]]
Intercept [-1.33855231]
Score 0.736318407960199
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


# Polynomial Feature Transformation

In [ ]:
def polynomial_regression(X_train, y_train, degree):
    poly_features = preprocessing.PolynomialFeatures(degree = degree, include_bias = False)
    print(X_train)
    X_poly = poly_features.fit_transform(X_train)
    return X_poly

In [ ]:
data = get_clean_data("cleaned_data.csv")
X_train, y_train = preprocess_normalize_data(data)
X_poly = polynomial_regression(X_train, y_train, 2)
# X_poly = polynomial_regression2(X_train, y_train)
coef, inter, score = train_logistic(X_poly, y_train)
print("Coefficients", coef)
print("Intercept", inter)
print("Score", score)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    htt

# Ridge Regularization

In [ ]:
def ridge_regularization(X_train, y_train, alpha):
    c = 1 /alpha
    logreg = LogisticRegression(C = c) # regular logistic regression, no regularization
    logreg.fit(X_train, y_train)
    return logreg.coef_, logreg.intercept_, logreg.score(X_train, y_train)

In [ ]:
data = get_clean_data("cleaned_data.csv")
X_train, y_train = preprocess_normalize_data(data)
coef, inter, score = ridge_regularization(X_train, y_train, 0.5)
# print("Coefficients", coef)
# print("Intercept", inter)
print("Score", score)

Score 0.7403367776502104
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
data = get_clean_data("cleaned_data.csv")
X_train, y_train = preprocess_normalize_data(data)
ridge_alpha_space = np.linspace(0,1,11)
for i in ridge_alpha_space:
    coef, inter, score = ridge_regularization(X_train, y_train, i)
    # print("Coefficients", coef)
    # print("Intercept", inter)
    print("Alpha", i, "Score", score)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwa

# Lasso Regularization

In [ ]:
def lasso_regularization(X_train, y_train, alpha):
    c = 1 /alpha
    logreg = LogisticRegression(C = c, penalty = 'l1', solver='liblinear') # regular logistic regression, no regularization
    logreg.fit(X_train, y_train)
    return logreg.coef_, logreg.intercept_, logreg.score(X_train, y_train)

In [ ]:
data = get_clean_data("cleaned_data.csv")
X_train, y_train = preprocess_normalize_data(data)
coef, inter, score = lasso_regularization(X_train, y_train, 0.5)
# print("Coefficients", coef)
# print("Intercept", inter)
print("Score", score)

Score 0.7455032529659396
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
data = get_clean_data("cleaned_data.csv")
X_train, y_train = preprocess_normalize_data(data)
ridge_alpha_space = np.linspace(0,1,11)
for i in ridge_alpha_space:
    coef, inter, score = lasso_regularization(X_train, y_train, i)
    # print("Coefficients", coef)
    # print("Intercept", inter)
    print("Alpha", i, "Score", score)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwa

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=90fab6ac-739b-4c97-b68c-ff3c09208031' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>